In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
import sys

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)

from openai import AzureOpenAI
from llama_index.llms import AzureOpenAI as LlamaIndexAzureOpenAI
from autocoder.telemetry import trace_client

from actionweaver.llms import patch
from langchain_community.utilities.github import GitHubAPIWrapper

from autocoder.bot import AutoCoder
from autocoder.index import RepositoryIndex
from autocoder.codebase import Codebase

os.environ["MODEL"]

'gpt-4-32k'

In [21]:
llm = trace_client(AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2023-10-01-preview",
))

llm_for_rag = LlamaIndexAzureOpenAI(
    engine=os.environ["MODEL"],
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2023-10-01-preview",
    use_azure_ad=False
)

In [22]:
github_repository = "TengHu/AutoCoder"
github_api = GitHubAPIWrapper(
    github_repository=github_repository,
    github_app_id=os.environ["GITHUB_APP_ID"],
    github_app_private_key=os.environ["GITHUB_APP_PRIVATE_KEY"],
)
codebase = Codebase(github_api)
index = RepositoryIndex(github_repository, codebase, llm_for_rag)

In [139]:
auto_coder = AutoCoder(index, codebase, llm, create_branch=False)
auto_coder.create_branch("demo")

[DEBUG] Branch 'demo_v8' created successfully, and set as current active branch.


"Branch 'demo_v8' created successfully, and set as current active branch."

In [140]:
input = '''[Code Change] Optimize the code in `autocoder/telemetry.py`
- refactor identity_decorator to be more compact
- remove redundant code in traceable and trace_client'''

In [141]:
res = auto_coder(input)
print(res)

[DEBUG] Planning code change: Optimize the code in `autocoder/telemetry.py`
- refactor identity_decorator to be more compact
- remove redundant code in traceable and trace_client
[DEBUG] Implementation plan created: Refactor the code in `autocoder/telemetry.py`, specifically the `identity_decorator`, `traceable` and `trace_client` functions. For `identity_decorator`, the function can be made more compact. For `traceable` and `trace_client`, redundant code should be removed.
autocoder/telemetry.py:1. For `identity_decorator`, it's currently a function that takes any function as input and returns the same function. This can be simplified. 
2. In `traceable`, there is a check if `LANGCHAIN_API_KEY` is in the environment variables and applies a decorator accordingly. This is repeated in `trace_client` as well. This is redundant and can be removed from one of the places.

[DEBUG] Modifying file: autocoder/telemetry.py
[DEBUG] Committing code change to autocoder/telemetry.py
  def identity_d

In [142]:
auto_coder("create PR")

'I have successfully created a new pull request with number 101.'